# Examine Nathan's kicker and inflector

Examine Nathan's new inflector and kicker classes and compare the fields with the old one. 

Deliveriables:
* Display of kicker and inflector fields
* Animation of kicker fields with time
* Maybe nice picture of some events

In [1]:
!pwd

/Users/lyon/dataAnalysisPlatform/Documents/g-2/project-20150529


Open a window to the right place to run stuff

In [1]:
%%file start-iTerm.scr
tell application "iTerm"
    activate
    set myterm to (make new terminal)
    tell myterm
        launch session "Default Session"
        tell the last session   
            set name to "project-20150529"
            write text "source /cvmfs/oasis.opensciencegrid.org/gm2/prod/g-2/setup"
            write text "cd ~/Development/g-2/projects/20150529/try/"
            write text "source ../localProducts_gm2_v6_01_00_debug/setup"
            write text "setup ninja v1_5_3a"
            write text ". mrb s"
            write text "export PATH=$PATH:/Applications/paraview4.3.1.app/Contents/bin"
        end tell
    end tell
end tell

Overwriting start-iTerm.scr


In [2]:
!osascript start-iTerm.scr

Kim says that the fcl files that actually work are in his directory at `/gm2/app/users/kkhaw/Work/kicker/FCL/`. I copied `laser.fcl` (pencil beam) and `realbeam.fcl` (Volodya's sample). There are lots of other fcl's there based on these. I think Kim must be scanning. I've asked him which file is best. 

Let's see what Nathan's code does...  Will be looking at `realbeam.fcl` (`laser.fcl`) is similar. 

The FCL loads `NathanInflector` and `NathanKicker`. 

Look at the kicker first:
* The electrodes are sensitive detectors and are turned into art records.
* `kPlates` are `989`, so the 989 field map is used (`g2RunTimeFiles/KickerBfield989.dat`).
* The kicker circuit defines the rise, top, and fall time.
* `KickerField::GetFieldValue` has the `useOneTurnApproximation` set - this means that the kicker is on full strength for the first turn of the muon around the ring and off for the rest. 


Note that the FCL's don't explicitly load the `TFileService`. If you supply the `-T <rootFileName>` option to `gm2`, then `TFileService` is loaded automatically. 

I ran with 
```bash
gm2 -c ./realbeam.fcl -n 5 -T inf_5events.root -o out_5events.root
```

Note that the default `realbeam.fcl` doesn't have the straw tracker in it.

Kim says to try his FCL file `realbeam_x-4.0_xp0.04_y0.0_yp0.0_dpp0.1_infXp-3.5_infBy0-1.4790_kBy0280.fcl` and Nathan says I can turn off the one turn approximation. I should set up a debug checker to see when the magnetic field is sampled and its values. Kim's FCL files are at `/gm2/app/users/kkhaw/Work/kicker/FCL/`. I've made a soft link in my `try` directory called `realbeam_good.fcl` to Kim's best FCL file above.

Let's try the following:
* Determine when and where Geant makes the kicker magnetic field requests. 

![Ring Map](ringMap.png)
(from https://muon.npl.washington.edu/elog/g2/General+Field+Team/86)

The above is a map of the ring. Note that Geant has the wrong Arc numbers. `S01` is Arc 11 and `S02` is Arc 00. So the kickers are in Arcs 02 and 03.

I have a breakpoint:
<img src="kickerOneTurnBP.png" width=400/>

The full text boxes are:
Script location: `/Users/lyon/Development/g-2/projects/20150529/try/magFieldCheck.sh`
and argument list `@Point[0]@,@Point[1]@,@Point[2]@,@Point[3]@,@TurnCounter::getInstance().turns()@,@Bfield[0]@,@Bfield[1]@,@Bfield[2]@`

In [11]:
!cat /Users/lyon/Development/g-2/projects/20150529/try/magFieldCheck.sh

#!/bin/sh

#  magFieldCheck.sh
#  project-20150529
#
#  Created by Adam Lyon on 6/7/15.
#  Copyright (c) 2015 Adam L Lyon. All rights reserved.

out="magFieldCheck.csv"

if [ ! -r "$out" ]; then
  echo 'x,y,z,t,nturns,Bx,By,Bz' > $out
fi

echo $1,$2,$3,$4,$5,$6,$7 >> $out

Trying with 
```bash
gm2 -c ./realbeam_good.fcl -n 5 -T inf_5events.root -o out_5events.root
```

<font color="red">Well</font> - there seems to be a buffer problem with writing out this information... I eventually get the deugger console file filling up with, 

```Error in shell command “/Users/lyon/Development/g-2/projects/20150529/try/magFieldCheck.sh” for breakpoint “gm2ringsim::KickerField::GetFieldValue( const G4double Point[4], G4double* Bfield )” - “Failed to set posix_spawn_file_actions for fd -1 at index 0 with errno 9”```

Output to the log seems to work much better! - But it's very slow! - But when I output more data, it seems to get stuck after awhile. I think this is just too slow and buggy to be useful. Write some code!!

## New Feature branch `feature/adamFieldDebug` based from `feature/nathan`

Adding a new main directory to `gm2analyses` called `debugging` -- This will contain visualization and debugging services.

Wrote `gm2ringsim/debugging/writeFieldLookup_service` which outputs to a file every field lookup.

Looking at the kicker field timing. The class that drives the kicker field is `KickerCircuit989Actual`. It reads a file to get the kicker map - but then it re-centers the pulse timing so that the peak is in a specific place. Complicated! The upshot is that the kicker turns on at -39ns, peaks at 132ns, and ends at 283ns. Note that different kicker magnets have different start/peak/end times. The numbers above are for kicker 0. 

Now we may want to do a field scan (moved to `debugging`) and do the scan with multiple parallel jobs. So we'll break up the scan in z (`k` iterator). You can now also enter the time the field is sampled in the FCL file. 

So I have a field scan of arcs 3 and 11, but the magnetic fields always comes out to be really small. For example, the units on the colors of the fields are in the $10^{-3}$. According to the TDR (section 9.7.2), the field is 1.45 T. When I run gm2 in the debugger, the storage ring field is $0.0014512990674790127$  (this comes from `gm2ringsim::arc::StorageRingField.cc:302`). Why is the value so small and why all the digits? -- Because the `tesla` unit in `CLHEP/Units/SystemOfUnits.h` is $0.001$ to convert to some standard unit of `0.001*megavolt*ns/mm2`. Have to debug more to see where the field itself is set. 

So anyway, to get tesla, should multiply values by 1000.

It would be nice to have a generic programmable source where you can use a selector to choose the file name.

Keeping track of what I'm doing... I added the `MetadataFromFile_module` to `gm2analyses/util`. I think the best way to make the file that records what I did is to run,

```bash
here=$PWD
ups active > $here/codeConfig.out
pushd $MRB_SOURCE
for i in `grep -v '^#' .cmake_add_subdir | cut -f 2 -d '(' | cut -f 1 -d ')'`; do 
   cd $i 
   echo "-----> $i" >> $here/codeConfig.out
   mrb c | head -1  >> $here/codeConfig.out
   cd  .. 
done
popd
```

##  Visualize muon path with and without kicker

See https://www.evernote.com/shard/s25/nl/2820802/6c36a84e-f1d0-4129-b30a-42b7589f0aa7/ for a general plan.